In [6]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
os.getcwd()
path = "/Users/yunho/projects/Tobigs_2018"

doc = pd.read_csv(os.path.join(path,"white.csv"))

In [7]:
# EDA
doc.head(6)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,y
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
5,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6


In [8]:
type(doc.y)

pandas.core.series.Series

In [9]:
# Data type of y -> str
doc.y= doc.y.astype("str")

In [11]:
# Check y distribution
doc.y.value_counts()

6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: y, dtype: int64

In [12]:
# Shape
doc.shape

(4898, 12)

In [13]:
# Split X, y
X_data=doc.iloc[:,0:11]
y_data = doc.y

In [14]:
# One-hot encoding for y
ohe = OneHotEncoder()
y_data1 = y_data.values
y_data1 = ohe.fit_transform(y_data1.reshape(-1, 1)).toarray()
y_data1

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
# Data partitioning
X_train, X_test, Y_train, Y_test = train_test_split(X_data, y_data1, test_size=0.2)

In [16]:
# 1.  sigmoid, GradientDescentOptimizer 
 
X = tf.placeholder(tf.float32,[None, 11])
Y = tf.placeholder(tf.float32,[None, 7])
 
# init
W1 = tf.Variable(tf.random_normal([11,7],stddev=0.01))
b1 = tf.Variable(tf.zeros([7]))
hypothesis = tf.sigmoid(tf.matmul(X, W1) + b1)
 
# cross entropy cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
# GradientDescentOptimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
 
# predicted value H(X) > 0.5 이면  true, otherwise false
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)

# calculate the average value of 0 or 1 using the number of epochs
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
 
# train
init = tf.global_variables_initializer()
sess= tf.Session()
sess.run(init)
 
# 10000 epochs
for step in range(10001):
    cost_val,  _ = sess.run([cost, optimizer], feed_dict={X:X_train, Y:Y_train})
    if step % 1000 == 0:
        print(step, '\t', cost_val)

0 	 1.9452848
1000 	 1.5538563
2000 	 1.5393864
3000 	 1.5364162
4000 	 1.5353813
5000 	 1.5346875
6000 	 1.5341562
7000 	 1.5337626
8000 	 1.5334461
9000 	 1.5331813
10000 	 1.5329592


In [17]:
# accuracy 27.9%
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
is_correct = tf.equal(tf.argmax(predicted,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))
print('accuracy : ', sess.run(accuracy, feed_dict={X: X_test, Y: Y_test}))

accuracy :  0.27959183


In [18]:
# 2.  Multi-layer neural network, sigmoid, AdamOptimizer 
 
X = tf.placeholder(tf.float32,[None, 11])
Y = tf.placeholder(tf.float32,[None, 7])
 
# 3-layer neural network
W1 = tf.Variable(tf.random_normal([11,10],stddev=0.01))
b1 = tf.Variable(tf.zeros([10]))
h1 = tf.sigmoid(tf.matmul(X, W1) + b1)
 
W2 = tf.Variable(tf.random_normal([10,10],stddev=0.01))
b2 = tf.Variable(tf.zeros([10]))
h2 = tf.sigmoid(tf.matmul(h1, W2) + b2)
 
W3 = tf.Variable(tf.random_normal([10,7],stddev=0.01))
b3 = tf.Variable(tf.zeros([7]))
hypothesis = tf.matmul(h2, W3) + b3
 
# cross entropy cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
# Adam
optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)
 
# predicted value H(X) > 0.5 is true, else false
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)

# calculate the average value of 0 or 1 using the number of epochs
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
 
# train
init = tf.global_variables_initializer()
sess= tf.Session()
sess.run(init)
 
# 10000 epochs
for step in range(10001):
    cost_val,  _ = sess.run([cost, optimizer], feed_dict={X:X_train, Y:Y_train})
    if step % 1000 == 0:
        print(step, '\t', cost_val)

0 	 1.9501402
1000 	 1.2856228
2000 	 1.2856228
3000 	 1.2856243
4000 	 1.2856834
5000 	 1.285623
6000 	 1.2856226
7000 	 1.2856196
8000 	 1.285624
9000 	 1.2856238
10000 	 1.2856265


In [19]:
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
is_correct = tf.equal(tf.argmax(predicted,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))
print('accuracy : ', sess.run(accuracy, feed_dict={X: X_test, Y: Y_test}))

accuracy :  0.27959183


In [20]:
#3. Xavier algorithm for weight initialization, Multi-layer neural network, Adamoptimizer, relu 
 
tf.reset_default_graph()
 
# 3-layer neural network
X = tf.placeholder(tf.float32,[None, 11])
Y = tf.placeholder(tf.float32,[None, 7])
 
W1 = tf.get_variable("W1",shape = [11,10],initializer = tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.zeros([10]))
h1 = tf.nn.relu(tf.matmul(X, W1) + b1)
 
W2 = tf.get_variable("W2",shape = [10,10],initializer = tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.zeros([10]))
h2 = tf.nn.relu(tf.matmul(h1, W2) + b2)
 
W3 = tf.get_variable("W3",shape = [10,7] ,initializer = tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.zeros([7]))
hypothesis = tf.matmul(h2, W3) + b3
 
# cross entropy cost function 
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
# Adam
optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)
 
# predicted value H(X) > 0.5 is true, else false
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
# calculate the average value of 0 or 1 using the number of epochs
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
 
# train
init = tf.global_variables_initializer()
sess= tf.Session()
sess.run(init)
 
# 10000 epochs
for step in range(10001):
    cost_val,  _ = sess.run([cost, optimizer], feed_dict={X:X_train, Y:Y_train})
    if step % 1000 == 0:
        print(step, '\t', cost_val)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

0 	 64.909874
1000 	 1.2856234
2000 	 1.2856234
3000 	 1.2856228
4000 	 1.2856228
5000 	 1.2856228
6000 	 1.2856236
7000 	 1.2856209
8000 	 1.2856208
9000 	 1.2856185
10000 	 1.2856218


In [21]:
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
is_correct = tf.equal(tf.argmax(predicted,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))
print('accuracy : ', sess.run(accuracy, feed_dict={X: X_test, Y: Y_test}))

accuracy :  0.27959183


In [22]:
# 4. Dropout, sigmoid, GradientDescentOptimizer, Multi-layer neural network
 
X = tf.placeholder(tf.float32,[None, 11])
Y = tf.placeholder(tf.float32,[None, 7])
keep_prob = tf.placeholder(tf.float32)
 
# 3-layer neural network
W1 = tf.Variable(tf.random_normal([11,30],stddev=0.01))
b1 = tf.Variable(tf.zeros([30]))
h1 = tf.sigmoid(tf.matmul(X, W1) + b1)
h1 = tf.nn.dropout(h1, keep_prob)
 
W2 = tf.Variable(tf.random_normal([30,256],stddev=0.01))
b2 = tf.Variable(tf.zeros([256]))
h2 = tf.sigmoid(tf.matmul(h1, W2) + b2)
h2 = tf.nn.dropout(h2, keep_prob)
 
W3 = tf.Variable(tf.random_normal([256,7],stddev=0.01))
b3 = tf.Variable(tf.zeros([7]))
hypothesis = tf.matmul(h2, W3) + b3
 
# cross entropy cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
# GradientDescentOptimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
 
# train
init = tf.global_variables_initializer()
sess= tf.Session()
sess.run(init)
 
# 10000 epochs - dropout 0.8 
for step in range(10001):
    cost_val,  _ = sess.run([cost, optimizer], feed_dict={X:X_train, Y:Y_train,keep_prob:0.8})
    if step % 1000 == 0:
        print(step, '\t', cost_val)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
0 	 1.912752
1000 	 1.2870835
2000 	 1.2867362
3000 	 1.286476
4000 	 1.2850051
5000 	 1.2809805
6000 	 1.2627347
7000 	 1.2429771
8000 	 1.2425663
9000 	 1.2446607
10000 	 1.2338846


In [23]:
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
is_correct = tf.equal(tf.argmax(predicted,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))
# No dropout for test
print('accuracy : ', sess.run(accuracy, feed_dict={X: X_test, Y: Y_test,keep_prob:1}))

accuracy :  0.27959183


In [25]:
#5. Dropout, relu, Multi-layer neural network, AdamOptimizer, Xavier algorithm for weight initialization
 
tf.reset_default_graph()
 
# 4-layer neural network
X = tf.placeholder(tf.float32,[None, 11])
Y = tf.placeholder(tf.float32,[None, 7])
keep_prob = tf.placeholder(tf.float32)
 
# Xavier algorithm for weight initialization
W1 = tf.get_variable("W1",shape = [11,20],initializer = tf.contrib.layers.xavier_initializer())
# bias
b1 = tf.Variable(tf.zeros([20]))
# relu 
h1 = tf.nn.relu(tf.matmul(X, W1) + b1)
# dropout
h1 = tf.nn.dropout(h1, keep_prob)
 

# Xavier algorithm for weight initialization 
W2 = tf.get_variable("W2",shape = [20,80],initializer = tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.zeros([80]))
h2 = tf.nn.relu(tf.matmul(h1, W2) + b2)
h2 = tf.nn.dropout(h2, keep_prob)
 
W3 = tf.get_variable("W3",shape = [80,30] ,initializer = tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.zeros([30]))
h3 = tf.nn.relu(tf.matmul(h2, W3) + b3)
h3 = tf.nn.dropout(h3, keep_prob)
 
W4 = tf.get_variable("W4",shape = [30,7] ,initializer = tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.zeros([7]))
h4 = tf.nn.dropout(h3, keep_prob)
 
hypothesis = tf.matmul(h4, W4) + b4
 
# cross entropy cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
# AdamOptimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)
 
# train
init = tf.global_variables_initializer()
sess= tf.Session()
sess.run(init)
 
# 10000 epochs - dropout 0.8 
for step in range(10001):
    cost_val,  _ = sess.run([cost, optimizer], feed_dict={X:X_train, Y:Y_train,keep_prob:0.8})
    if step % 1000 == 0:
        print(step, '\t', cost_val)

0 	 22.76609
1000 	 1.2856231
2000 	 1.2856231
3000 	 1.2856228
4000 	 1.2856234
5000 	 1.2856231
6000 	 1.2856194
7000 	 1.2856222
8000 	 1.2856232
9000 	 1.2856221
10000 	 1.2856295


In [26]:
# test
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
is_correct = tf.equal(tf.argmax(predicted,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))
print('accuracy : ', sess.run(accuracy, feed_dict={X: X_test, Y: Y_test,keep_prob:1}))

accuracy :  0.27959183
